In [ ]:
!pip install lime
!pip install transformers
!pip install pytreebank
!pip install datasets==1.5.0

import os
import random

import pandas as pd
import numpy as np
import scipy as sp
import torch
import spacy
from torch.utils.data import \
    TensorDataset, \
    DataLoader
from transformers import \
    BertTokenizer, \
    BertForSequenceClassification, \
    AdamW, \
    BertConfig, \
    get_linear_schedule_with_warmup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/NLP Project'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Load your data**

In [ ]:

import src.data.dataload
sst=src.data.dataload.load_sst()
train, val, test = sst.train_val_test

In [ ]:
ag_news=src.data.dataload.load_agnews()
train_ag, _, _ = ag_news.train_val_test

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a. Subsequent calls will reuse this data.


**Set up your model & tokenizer**

In [ ]:
'''
setup your model & tokenizer
'''
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP Project/BERT Model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

id2label = bert_model.config.id2label
label2id = bert_model.config.label2id
labels = sorted(label2id, key=label2id.get)

**Explanations on SST**

In [ ]:
'''
produce explanations on some data
'''
from lime.lime_text import LimeTextExplainer
LIME_explainer = Explainer(predict_proba,labels,'LIME')

indices=np.random.choice(len(test), 20, replace=False)
instance_array = test['sentencee'].iloc[indices]

top_tokens_SST,top_values_SST=LIME_explainer.explain_instances(instance_array)

In [ ]:
top_tokens,top_values

**Explanations on AG News**

In [ ]:
indices=np.random.choice(len(test), 20, replace=False)
instance_array = train_ag['text'].iloc[indices]
top_tokens_AG,top_values_AG=LIME_explainer.explain_instances(instance_array)

In [ ]:
class Explainer():

  def __init__(self, predict_proba,labels, explainer_type='LIME'):
    '''
    predict_proba - predict function which will depend on model type
    '''
    if explainer_type == 'LIME':
      self.exp = LimeTextExplainer(class_names=labels)

    self.predict_proba=predict_proba

  def explain_instance(self,x):
    '''
    x - 1 input instance
    
    returns - list of top tokens/importance weights
    '''
    exp_instance=self.exp.explain_instance(x, self.predict_proba, num_features=10,top_labels=5,num_samples=50)

    pred_label = np.argmax(exp_instance.predict_proba)

    top_tokens=[x[0] for x in exp_instance.as_list(label=pred_label)]
    top_values = [x[1] for x in exp_instance.as_list(label=pred_label)]

    return top_tokens,top_values

  def explain_instances(self,X):
    '''
    X - array of input sentences
    '''

    top_tokens_list=[]
    top_values_list = []

    for s in X:

      top_tokens,top_values = self.explain_instance(s)

      top_tokens_list.append(top_tokens)
      top_values_list.append(top_values)

    return top_tokens_list,top_values_list


def predict_proba_BERT(x):

  '''
  this depends on the model, will be passed to explainer
  '''

  if isinstance(x,str):
    x=[x]

  with torch.no_grad():
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128,truncation=True) for v in x]).to(device)
    attention_mask = (tv!=0).type(torch.int64).to(device)
    outputs = bert_model(tv,attention_mask=attention_mask)
    scores = torch.softmax(outputs[0],dim=1)

    return scores.cpu().detach().numpy()

def predict_proba_BCN(x):

  #predict only on the sentence
  title = ' '

  a = predictor.predict_batch_json([
      dict(title=title, Description=s) for s in x
  ])

  class_probs=np.array([t['class_probabilities'] for t in a])
  return class_probs